In [2]:
# Skaičiuoja, kiek Zive įrašuose yra rpeaks, nesutampančių tarp tų, kurie rasti
# grynai automatiniu būdu ir tų, kurie yra pataisyti anotuojant įrašus.   

import neurokit2 as nk
import numpy as np
from pathlib import Path
import os, sys, json
from zive_util_vu import zive_read_df_rpeaks, get_filename


def misalignments(rec_dir, SubjCode):

       # I-a dalis: nuskaitome rpeaks iš json failo
    df_rpeaks = zive_read_df_rpeaks(rec_dir, str(SubjCode))
    atr_sample = df_rpeaks['sampleIndex'].to_numpy()
    
    # print(atr_sample[:5])

        # II-a dalis: suformuojame rpeaks su Neurokitu
    file_path = Path(rec_dir, str(SubjCode) + '.npy')
    signal_raw = np.load(file_path)
    _, rpeaks = nk.ecg_peaks(signal_raw, sampling_rate=200, correct_artifacts=False)
    rpeaks_from_signal = rpeaks['ECG_R_Peaks']
    # print(rpeaks_from_signal[:5])
  
        # Sulyginimas
    ab = np.setdiff1d(atr_sample, rpeaks_from_signal)
    ba = np.setdiff1d(rpeaks_from_signal,atr_sample)
    return(np.size(ab)+np.size(ba))


my_os=sys.platform
print("OS in my system : ",my_os)

if my_os != 'linux':
    OS = 'Windows'
else:  
    OS = 'Ubuntu'
# Bendras duomenų aplankas, kuriame patalpintas subfolderis name_db

if OS == 'Windows':
    Duomenu_aplankas = 'D:\\DI'   # variantas: Windows, GUNDAS
    # Duomenu_aplankas = 'F:\DI\Data\MIT&ZIVE\VU'   # variantas: Windows, HERKULIS
else:
    Duomenu_aplankas = '/home/kesju/DI'   # arba variantas: UBUNTU, be Docker

# jei variantas Docker pasirenkame:
# Duomenu_aplankas = '/Data/MIT&ZIVE'

# Vietinės talpyklos aplankas ir pūpsnių atributų failas
db_folder = 'DUOM_2022_RUDUO'

#  Nuoroda į aplanką su MIT2ZIVE duomenų rinkiniu
db_path = Path(Duomenu_aplankas, db_folder)

# Nuoroda į aplanką su EKG įrašais (.npy) ir anotacijomis (.json)
rec_dir = Path(db_path, 'records_npy')

# Variantas: visi duomenys
# Nuskaitome failą info_create.json ir duomenų rinkinio parametrus
file_path = Path(rec_dir,'info_create_z.json')
with open(file_path) as json_file:
    info_create = json.load(json_file)
SubjCodes =  info_create['SubjCodes'] # pacientų įrašų sąrašas

# Variantas: testinė imtis  
# file_path = Path(rec_dir, 'test_subjcode_lst.csv')
# SubjCodes = list(np.loadtxt(file_path, delimiter=',', dtype="int"))

tot, ns_tot = 0, 0

for SubjCode in SubjCodes:
    ns = misalignments(rec_dir, SubjCode)
    filename = get_filename(rec_dir, SubjCode)
    print(f"Zive įrašas: {SubjCode} filename: {filename} Nesutampančių rpeaks skaičius ns: {ns} ")
    if (ns != 0):
        ns_tot +=1
    tot +=1
print(f"\n Viso įrašų: {tot}  Įrašų su nesutampančiais rpeaks: {ns_tot} ({100*ns_tot/tot:.1f}%)")        

 

ModuleNotFoundError: No module named 'zive_util_vu'